# Vunlerable model train _MNIST

In [1]:
# mnist dataset
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.optimizers as optim
import numpy as np

In [2]:
import os
root_logdir = os.path.join(os.curdir,"mylogs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir,run_id)

In [3]:
#load mnist data
mnist = keras.datasets.mnist
(X_train_full,y_train_full),(X_test,y_test) = mnist.load_data()

#data split
X_valid,X_train = X_train_full[:5000]/255.0,X_train_full[5000:]/255.0
y_valid,y_train = y_train_full[:5000],y_train_full[5000:]
X_train = np.expand_dims(X_train,-1)
X_valid = np.expand_dims(X_valid,-1)
X_test = np.expand_dims(X_test,-1)

num_classes=10

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
model = keras.Sequential(
    [
        keras.Input(shape=X_train.shape[1:]),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dense(200,activation="relu"),
        layers.Dense(200,activation="relu"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ]
)

batch_size = 128
epochs = 30

run_logdir = get_run_logdir()
history = model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
checkpoint_cb = keras.callbacks.ModelCheckpoint("cnn_Mnist_model.h5",save_best_only=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
# model.summary()

# MNIST vunlerable model

In [6]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,\
          validation_data=(X_valid,y_valid),\
          callbacks=[tensorboard_cb,checkpoint_cb])

Epoch 1/30
430/430 [==============================] - 5s 6ms/step - loss: 0.5366 - accuracy: 0.8199 - val_loss: 0.0611 - val_accuracy: 0.9828
Epoch 2/30
430/430 [==============================] - 2s 4ms/step - loss: 0.0692 - accuracy: 0.9776 - val_loss: 0.0431 - val_accuracy: 0.9884
Epoch 3/30
430/430 [==============================] - 2s 4ms/step - loss: 0.0472 - accuracy: 0.9857 - val_loss: 0.0436 - val_accuracy: 0.9886
Epoch 4/30
430/430 [==============================] - 2s 4ms/step - loss: 0.0368 - accuracy: 0.9889 - val_loss: 0.0307 - val_accuracy: 0.9914
Epoch 5/30
430/430 [==============================] - 2s 3ms/step - loss: 0.0303 - accuracy: 0.9907 - val_loss: 0.0429 - val_accuracy: 0.9890
Epoch 6/30
430/430 [==============================] - 1s 3ms/step - loss: 0.0298 - accuracy: 0.9907 - val_loss: 0.0373 - val_accuracy: 0.9896
Epoch 7/30
430/430 [==============================] - 1s 3ms/step - loss: 0.0230 - accuracy: 0.9927 - val_loss: 0.0315 - val_accuracy: 0.9922
Epoch 

In [7]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 12.2961 - accuracy: 0.9887


[12.296073913574219, 0.9886999726295471]

# CIFAR10 vunlerable train

In [54]:
cifar10 = keras.datasets.cifar10
(X_train,y_train),(X_test,y_test) = cifar10.load_data()

In [55]:
# X_valid,X_train = X_train_full[:5000]/255.0,X_train_full[5000:]/255.0
# y_valid,y_train = y_train_full[:5000],y_train_full[5000:]
# X_train = np.expand_dims(X_train,-1)
# X_valid = np.expand_dims(X_valid,-1)
# X_test = np.expand_dims(X_test,-1)
X_train = X_train/255.0
class_names = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

num_classes=10
# convert class vectors to binary class matrices

y_train= keras.utils.to_categorical(y_train, num_classes)
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_valid = keras.utils.to_categorical(y_valid, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [56]:
y_train_full.shape

(50000, 10)

In [22]:
# resnet50
baseModel = tf.keras.applications.resnet.ResNet50(\
    include_top=False,\
    weights='imagenet',\
    input_tensor=None,\
    input_shape=[32,32,3],\
    pooling=None,\
    classes=10\
)
for layer in baseModel.layers:
    layer.trainable = True

headModel = baseModel.output
headModel = layers.Flatten(name="flatten")(headModel)
headModel = layers.Dropout(0.5)(headModel)
headModel = layers.Dense(10, activation="softmax")(headModel)

batch_size = 128
epochs = 5
model = keras.Model(inputs=baseModel.input, outputs=headModel)
#tensorboard init
run_logdir = get_run_logdir()
model.compile(loss="categorical_crossentropy", optimizer=optim.Adam(learning_rate=1e-3), metrics=["accuracy"])
checkpoint_cb = keras.callbacks.ModelCheckpoint("resnet50_cifar10_model.h5",save_best_only=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [57]:
#train cifar10 resnet50
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,\
          validation_split=0.1,\
          callbacks=[tensorboard_cb,checkpoint_cb])

Epoch 1/30


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [48]:
model.predict(X_train_full)[:10].round(1)

array([[0. , 0. , 0. , 0.3, 0. , 0.5, 0.1, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9],
       [0. , 0. , 0. , 0.1, 0. , 0. , 0.1, 0.1, 0. , 0.6],
       [0. , 0. , 0. , 0.1, 0.7, 0.3, 0. , 0. , 0. , 0. ],
       [0.1, 0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.4],
       [0. , 0. , 0. , 0.2, 0. , 0.1, 0.3, 0.2, 0. , 0.1],
       [0. , 0. , 0.1, 0. , 0.6, 0.1, 0. , 0.1, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ],
       [0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0. ],
       [0. , 0. , 0. , 0.7, 0. , 0.1, 0. , 0. , 0. , 0. ]], dtype=float32)

In [49]:
y_train_full[:10]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [69]:
model.evaluate(X_train,y_train)

1407/1407 [==============================] - 2s 1ms/step - loss: 0.7424 - accuracy: 0.7415


[0.7423765659332275, 0.7414666414260864]

In [63]:
pred = model.predict(X_test)

In [68]:
print(y_test[2])
print(pred[2])

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0622445e-20 0.0000000e+00]


In [86]:
baseModel.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________